In [92]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict, Counter
from math import log
import entropy
import pdb

In [ ]:
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
# dest = pd.read_csv('destinations.csv')

In [ ]:
# n = 37000000
# s = 1000
# skip = sorted(random.sample(xrange(n),n-s))
# df = pd.read_csv('train.csv', skiprows=skip)

In [54]:
df_train = pd.read_csv('train.csv', nrows=10000)

In [55]:
df_train.describe()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,6271.000000,10000.000000,10000.000000,10000.000000,10000.00000,...,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8.261400,2.744400,85.711700,272.578700,26941.592000,1833.809641,29121.770900,0.206600,0.220100,5.70210,...,0.349100,1.12780,14807.539200,2.641100,0.077700,1.456100,3.149200,80.198000,575.825700,49.120100
std,10.352004,0.590511,61.545901,178.451546,17472.051101,2028.265606,17115.773996,0.404886,0.414335,3.59959,...,0.761241,0.55308,11454.009352,2.203446,0.267712,1.167565,1.539213,55.267289,494.452768,28.705492
min,2.000000,0.000000,0.000000,0.000000,3.000000,0.065100,12.000000,0.000000,0.000000,0.00000,...,0.000000,0.00000,11.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,3.000000,66.000000,172.000000,9876.000000,261.557950,12950.000000,0.000000,0.000000,2.00000,...,0.000000,1.00000,8268.000000,1.000000,0.000000,1.000000,2.000000,50.000000,142.000000,25.000000
50%,2.000000,3.000000,66.000000,258.000000,25954.000000,1210.896700,30977.000000,0.000000,0.000000,9.00000,...,0.000000,1.00000,10970.500000,1.000000,0.000000,1.000000,2.000000,50.000000,544.000000,48.000000
75%,11.000000,3.000000,69.000000,354.000000,43175.000000,2397.099800,44663.000000,0.000000,0.000000,9.00000,...,0.000000,1.00000,18714.000000,5.000000,0.000000,1.000000,4.000000,105.000000,700.000000,73.000000
max,47.000000,4.000000,239.000000,1003.000000,56451.000000,11301.871400,54404.000000,1.000000,1.000000,9.00000,...,9.000000,8.00000,65035.000000,8.000000,1.000000,23.000000,6.000000,208.000000,2103.000000,99.000000


In [78]:
df_train.columns

Index([u'date_time', u'site_name', u'posa_continent', u'user_location_country',
       u'user_location_region', u'user_location_city',
       u'orig_destination_distance', u'user_id', u'is_mobile', u'is_package',
       u'channel', u'srch_ci', u'srch_co', u'srch_adults_cnt',
       u'srch_children_cnt', u'srch_rm_cnt', u'srch_destination_id',
       u'srch_destination_type_id', u'is_booking', u'cnt', u'hotel_continent',
       u'hotel_country', u'hotel_market', u'hotel_cluster'],
      dtype='object')

In [57]:
df_test = pd.read_csv('test.csv', nrows=10000)

In [58]:
df_test.describe()

,id,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market
count,10000.00000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,5893.000000,10000.000000,10000.000000,10000.00000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,4999.50000,11.58600,2.535700,74.226700,294.917700,27006.775500,1723.991252,16378.395600,0.138600,0.10910,6.21540,1.949500,0.228400,1.119000,16375.030700,2.855300,3.112500,83.942200,632.60590
std,2886.89568,12.03826,0.844865,58.763332,218.317558,17141.926348,2274.012556,9422.853533,0.345546,0.31178,3.92267,0.901682,0.645659,0.454158,12788.207489,2.258467,1.692079,54.428304,516.74026
min,0.00000,2.00000,0.000000,0.000000,0.000000,14.000000,0.007400,1.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,8.000000,1.000000,0.000000,0.000000,1.00000
25%,2499.75000,2.00000,2.000000,66.000000,154.750000,10494.250000,207.768100,8301.000000,0.000000,0.00000,2.00000,1.000000,0.000000,1.000000,8267.000000,1.000000,2.000000,50.000000,212.00000
50%,4999.50000,2.00000,3.000000,66.000000,260.000000,26233.000000,683.431800,16544.000000,0.000000,0.00000,9.00000,2.000000,0.000000,1.000000,11951.000000,1.000000,2.000000,50.000000,609.50000
75%,7499.25000,24.00000,3.000000,69.000000,385.000000,42300.000000,2269.053000,24233.750000,0.000000,0.00000,10.00000,2.000000,0.000000,1.000000,22983.000000,6.000000,4.000000,106.000000,816.50000
max,9999.00000,52.00000,4.000000,239.000000,1017.000000,56507.000000,11631.339100,33201.000000,1.000000,1.00000,10.00000,9.000000,6.000000,8.000000,65768.000000,9.000000,6.000000,212.000000,2116.00000


In [59]:
df_test.columns

Index([u'id', u'date_time', u'site_name', u'posa_continent',
       u'user_location_country', u'user_location_region',
       u'user_location_city', u'orig_destination_distance', u'user_id',
       u'is_mobile', u'is_package', u'channel', u'srch_ci', u'srch_co',
       u'srch_adults_cnt', u'srch_children_cnt', u'srch_rm_cnt',
       u'srch_destination_id', u'srch_destination_type_id', u'hotel_continent',
       u'hotel_country', u'hotel_market'],
      dtype='object')

In [60]:
set(df_train.columns) ^ set(df_test.columns) 

{'cnt', 'hotel_cluster', 'id', 'is_booking'}

In [61]:
def nan_analysis(_df):
    # columns with missing values
    columns_w_missing_values = _df.columns[_df.isnull().any()]
    for col in columns_w_missing_values:
        print '# of nan values in column {0}: {1}'.format(col, sum(1 if x else 0 for x in _df[col].isnull()))

In [62]:
nan_analysis(df_train)
nan_analysis(df_test)

# of nan values in column orig_destination_distance: 3729
# of nan values in column srch_ci: 7
# of nan values in column srch_co: 7
# of nan values in column orig_destination_distance: 4107


In [68]:
labels = df_train['hotel_cluster'].values
labels_freq = Counter(labels)

In [76]:
total = float(sum(labels_freq.values()))
for k,v in sorted(labels_freq.items(), key=lambda x: x[1], reverse=True)[:20]:
    print '{0}: {1}'.format(k, v / total * 100)

91: 2.57
41: 2.37
65: 2.27
48: 2.04
25: 1.88
33: 1.81
95: 1.63
18: 1.6
21: 1.56
70: 1.54
16: 1.44
40: 1.42
83: 1.42
12: 1.38
8: 1.37
36: 1.35
28: 1.34
57: 1.33
97: 1.33
52: 1.32


In [97]:
def entropy(labels):
    """ Computes entropy of label distribution. """
    n_labels = len(labels)

    if n_labels <= 1:
        return 0
        
    counts = np.bincount(labels)
    probs = counts / float(n_labels)
    n_classes = np.count_nonzero(probs)
    
    if n_classes <= 1:
        return 0
    
    ent = 0.

    # Compute standard entropy.
    log_function = log
    for i in probs:
        pdb.set_trace()
        ent -= i * log_function(i, n_classes)

    return ent

In [102]:
# entropy(df_train.site_name)
import entropy
entropy.shannon_entropy(df_train.site_name.values.tobytes())

0.09752917357606962

In [152]:
low_entropy = np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
med_entropy = np.array([1,0,0,1,1])
high_entropy = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
range_entropy = np.array(range(10))

In [153]:
entropy.shannon_entropy(low_entropy)

0.005676835699730627

In [154]:
entropy.shannon_entropy(med_entropy)

0.04803894065432304

In [155]:
entropy.shannon_entropy(high_entropy)

0.06794555518364308

In [156]:
entropy.shannon_entropy(range_entropy)

0.10800344286424633